In [1]:
import pandas as pd

In [2]:
rules = pd.read_csv("all-rules-with-metrics.csv")
possible_rules = set(zip(rules[rules["isPossible"]]["fromLib"], rules[rules["isPossible"]]["toLib"]))
confirmed_rules = set(zip(rules[rules["isConfirmed"]]["fromLib"], rules[rules["isConfirmed"]]["toLib"]))

In [3]:
rules

,fromLib,toLib,ruleFreq,concurrence,addedTogether,distance,apiCounter,isPossible,isConfirmed,relativeRuleFrequency,concurrenceAdjustment,popularityRegularization,apiSupport,distanceAdjustment
0,antlr:antlr,commons-collections:commons-collections,82,726,666,198;10;316;6;8;7;185;3;8;8;8;8;4;8;8;7;8;8;8;7...,0,False,False,1.000000,0.112948,0.112948,0.0,1.290075
1,antlr:antlr,junit:junit,75,708,538,0;10;2;8;7;3;8;8;8;8;8;4;8;8;7;8;8;8;7;8;5;8;6...,0,False,False,0.914634,0.105932,0.096889,0.0,1.527907
2,antlr:antlr,org.slf4j:slf4j-api,67,622,486,10;6;8;7;3;8;8;8;8;4;8;8;7;8;8;8;7;8;8;5;8;8;8...,0,False,False,0.817073,0.107717,0.088013,0.0,1.437399
3,antlr:antlr,dom4j:dom4j,61,620,594,146;10;316;6;8;7;185;3;6;2;1;4;6;3;1;1;1;1;1;1...,0,False,False,0.743902,0.098387,0.073190,0.0,1.227770
4,antlr:antlr,commons-io:commons-io,56,478,388,1;181;3;8;8;8;8;8;4;8;8;7;8;8;8;7;8;5;8;8;1;42...,0,False,False,0.682927,0.117155,0.080008,0.0,1.226968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653304,xstream:xstream,org.antlr:antlr-runtime,1,14,2,5,0,False,False,0.029412,0.071429,0.002101,0.0,1.000000
653305,xstream:xstream,org.drools:drools-ant,1,2,0,1,0,False,False,0.029412,0.500000,0.014706,0.0,3.000000
653306,xstream:xstream,jboss.seam.profiles:seam-nopersistence,1,2,0,5,0,False,False,0.029412,0.500000,0.014706,0.0,1.000000
653307,xstream:xstream,javax.el:el-ri,1,4,0,3,0,False,False,0.029412,0.250000,0.007353,0.0,1.500000


We use the following metrics
1. Top@k precision
2. Top@k recall
3. Normalized Distributed Cumulative Gain
4. Mean Reciporical Rank

In [12]:
def evaluate(method, rules, possible_rules, confirmed_rules):
    top_k = 10
    top_rules = [list() for x in range(0, top_k)]
    from_lib_set = set(x for x, y in confirmed_rules)
    for from_lib, candidates in rules.groupby(by="fromLib"):
        if from_lib not in from_lib_set:
            continue
        #if all(isConfirmed is False or ruleFreq < 25 for isConfirmed, ruleFreq in zip(candidates["isConfirmed"], candidates["ruleFreq"])):
            #continue
        this_rules = [(from_lib, to_lib) for to_lib in candidates["toLib"]]
        for k, (from_lib, to_lib) in enumerate(this_rules):
            if k >= top_k:
                continue
            top_rules[k].append((from_lib, to_lib))
    for k in range(1, top_k):
        top_rules[k] += top_rules[k - 1]
    print("Result of {}:".format(method))
    for k in range(0, top_k):
        precision = len([x for x in top_rules[k] if x in confirmed_rules]) / len(top_rules[k])
        recall = len([x for x in top_rules[k] if x in confirmed_rules]) / len(confirmed_rules)
        f_measure = 2 * precision * recall / (precision + recall)
        precision_possible = len([x for x in top_rules[k] if x in possible_rules]) / len(top_rules[k])
        recall_possible = len([x for x in top_rules[k] if x in possible_rules]) / len(possible_rules)
        f_measure_possible = 2 * precision_possible * recall_possible / (precision_possible + recall_possible)
        print("Top {:2}: Precision-C/P = {:0.3f}/{:0.3f}, Recall-C/P = {:0.3f}/{:0.3f}, F-Measure-C/P = {:0.3f}/{:0.3f}, NDCG = {}, MRR = {}"
              .format(k + 1, precision, precision_possible, recall, recall_possible, f_measure, f_measure_possible, 0, 0))
def filter_and_sort1(rules):
    return rules[rules["ruleFreq"] > 25].sort_values(by=["fromLib", "ruleFreq"], ascending=[True, False])
def teyton2(rules):
    return rules[rules["relativeRuleFrequency"] > 0.07].sort_values(by=["fromLib", "ruleFreq"], ascending=[True, False])
def teyton(rules):
    return rules.sort_values(by=["fromLib", "relativeRuleFrequency"], ascending=[True, False])
def our_method(rules):
    rules["confidence"] = (rules["relativeRuleFrequency"] * (rules["concurrenceAdjustment"] ** 0.5)
                           * rules["apiSupport"].apply(lambda x: max(0.2, x)) ** 0.5
                           * rules["distanceAdjustment"] ** 2)
    return rules[rules["concurrence"] >= 10].sort_values(by=["fromLib", "confidence"], ascending=[True, False])
evaluate("Our Method", our_method(rules), possible_rules, confirmed_rules)

Result of Our Method:
Top  1: Precision-C/P = 0.247/0.258, Recall-C/P = 0.080/0.016, F-Measure-C/P = 0.120/0.030, NDCG = 0, MRR = 0
Top  2: Precision-C/P = 0.253/0.263, Recall-C/P = 0.163/0.032, F-Measure-C/P = 0.198/0.058, NDCG = 0, MRR = 0
Top  3: Precision-C/P = 0.219/0.240, Recall-C/P = 0.211/0.044, F-Measure-C/P = 0.215/0.075, NDCG = 0, MRR = 0
Top  4: Precision-C/P = 0.194/0.218, Recall-C/P = 0.249/0.054, F-Measure-C/P = 0.218/0.086, NDCG = 0, MRR = 0
Top  5: Precision-C/P = 0.181/0.204, Recall-C/P = 0.291/0.063, F-Measure-C/P = 0.223/0.096, NDCG = 0, MRR = 0
Top  6: Precision-C/P = 0.174/0.203, Recall-C/P = 0.336/0.075, F-Measure-C/P = 0.229/0.109, NDCG = 0, MRR = 0
Top  7: Precision-C/P = 0.155/0.183, Recall-C/P = 0.349/0.079, F-Measure-C/P = 0.215/0.110, NDCG = 0, MRR = 0
Top  8: Precision-C/P = 0.151/0.180, Recall-C/P = 0.388/0.089, F-Measure-C/P = 0.217/0.119, NDCG = 0, MRR = 0
Top  9: Precision-C/P = 0.143/0.174, Recall-C/P = 0.415/0.097, F-Measure-C/P = 0.213/0.124, NDCG =

In [13]:
our_method(rules).to_csv("test.csv", index=False)

In [14]:
evaluate("Teyton et al in JSS", teyton(rules), possible_rules, confirmed_rules)
evaluate("Teyton et al in WCRE", teyton2(rules), possible_rules, confirmed_rules)
evaluate("Sort by ruleFreq, keep ruleFreq > 25", filter_and_sort1(rules), possible_rules, confirmed_rules)

Result of Teyton et al in JSS:
Top  1: Precision-C/P = 0.032/0.053, Recall-C/P = 0.010/0.003, F-Measure-C/P = 0.016/0.006, NDCG = 0, MRR = 0
Top  2: Precision-C/P = 0.059/0.085, Recall-C/P = 0.038/0.011, F-Measure-C/P = 0.046/0.019, NDCG = 0, MRR = 0
Top  3: Precision-C/P = 0.082/0.124, Recall-C/P = 0.080/0.023, F-Measure-C/P = 0.081/0.039, NDCG = 0, MRR = 0
Top  4: Precision-C/P = 0.096/0.138, Recall-C/P = 0.125/0.034, F-Measure-C/P = 0.108/0.055, NDCG = 0, MRR = 0
Top  5: Precision-C/P = 0.090/0.136, Recall-C/P = 0.145/0.042, F-Measure-C/P = 0.111/0.065, NDCG = 0, MRR = 0
Top  6: Precision-C/P = 0.085/0.130, Recall-C/P = 0.166/0.048, F-Measure-C/P = 0.113/0.070, NDCG = 0, MRR = 0
Top  7: Precision-C/P = 0.081/0.124, Recall-C/P = 0.183/0.054, F-Measure-C/P = 0.112/0.075, NDCG = 0, MRR = 0
Top  8: Precision-C/P = 0.078/0.116, Recall-C/P = 0.201/0.058, F-Measure-C/P = 0.112/0.077, NDCG = 0, MRR = 0
Top  9: Precision-C/P = 0.071/0.109, Recall-C/P = 0.208/0.061, F-Measure-C/P = 0.106/0.07